In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv 
import scipy
from scipy.sparse import csr_matrix
from scipy.sparse import diags

def put_pts_on_square(xmin,xmax,n):
    
    t = np.linspace(xmin,xmax,n+1)
    pts1 = np.zeros((n+1,2))
    pts1[:,0] = t
    pts1[:,1] = xmin
    
    pts2 = np.zeros((n+1,2))
    pts2[:,0] = t
    pts2[:,1] = xmax
    
    pts3 = np.zeros((n+1,2))
    pts3[:,0] = xmin
    pts3[:,1] = t
    
    pts4 = np.zeros((n+1,2))
    pts4[:,0] = xmax
    pts4[:,1] = t
    
    return pts1, pts2, pts3, pts4

def reparametrization(path,h):
    dp = path - np.roll(path,1,axis = 0)
    dp[0,:] = 0;
    dl = np.sqrt(np.sum(dp**2,axis=1))
    lp = np.cumsum(dl);
    length = lp[-1];
    lp = lp/length; # normalize
    npath = int(round(length/h));
    g1 = np.linspace(0,1,npath)
    path_x = np.interp(g1,lp,path[:,0])
    path_y = np.interp(g1,lp,path[:,1])
    path = np.zeros((npath,2))
    path[:,0] = path_x
    path[:,1] = path_y
    return path

def find_bdry_pts_rec_ver(pts,xc,h0):
    ind = np.argwhere( np.abs(pts[:,0]-xc) < h0*1e-2) 
    Nind = np.size(ind)
    ind = np.reshape(ind,(Nind,))
    return Nind,ind

def find_bdry_pts_rec_hor(pts,yc,h0):
    ind = np.argwhere( np.abs(pts[:,1]-yc) < h0*1e-2)
    Nind = np.size(ind)
    ind = np.reshape(ind,(Nind,))
    return Nind,ind

def circulant_matrix(v):
    n = len(v)
    c_matrix = np.zeros((n, n), dtype=v.dtype)
    for i in range(n):
        c_matrix[i, :] = np.roll(v, i)
    return c_matrix

def stima3(verts):
    
    Aux = np.ones((3,3))
    Aux[1:3,:] = np.transpose(verts)
    
    rhs = np.zeros((3,2))
    rhs[1,0] = 1
    rhs[2,1] = 1
    
    G = np.zeros((3,2))
    G[:,0] = np.linalg.solve(Aux,rhs[:,0])
    G[:,1] = np.linalg.solve(Aux,rhs[:,1])
    
    M = 0.25*np.linalg.det(Aux)*np.matmul(G,np.transpose(G))
    return M

def massma3(verts):
    
    Aux = np.ones((3,3))
    Aux[1:3,:] = np.transpose(verts)
    
    v = np.array([2, 1, 1])           # Input vector
    H = circulant_matrix(v)           # Generate circulant matrix
    
    M = (1/24)*np.linalg.det(Aux)*H
    
    return M

def ucube(verts, U):
    
    Aux = np.ones((3,3))
    Aux[1:3,:] = np.transpose(verts)
    
    nonlinear_matrix = np.array([
    [4 * U[0]**3 + U[1]**3 + U[2]**3 + 3 * U[0]**2 * (U[1] + U[2]) + 2 * U[0] * (U[1]**2 + U[2]**2) + U[1] * U[2] * (U[1] + U[2]) + 2 * U[0] * U[1] * U[2]],
    [4 * U[1]**3 + U[0]**3 + U[2]**3 + 3 * U[1]**2 * (U[0] + U[2]) + 2 * U[1] * (U[0]**2 + U[2]**2) + U[0] * U[2] * (U[0] + U[2]) + 2 * U[0] * U[1] * U[2]],
    [4 * U[2]**3 + U[1]**3 + U[0]**3 + 3 * U[2]**2 * (U[1] + U[0]) + 2 * U[2] * (U[1]**2 + U[0]**2) + U[1] * U[0] * (U[1] + U[0]) + 2 * U[0] * U[1] * U[2]]
    ]).reshape(3,1)
    
    M = (1/120)*np.linalg.det(Aux)*nonlinear_matrix
    
    return M

def ucube_linear(verts, U):
    
    Aux = np.ones((3,3))
    Aux[1:3,:] = np.transpose(verts)
    
    linear_nonlinear_matrix = np.array([
    [12 * U[0]**2 + 2 * (U[1]**2 + U[2]**2 + U[1] * U[2]) + 6 * U[0] * (U[1] + U[2]),
     3 * (U[0]**2 + U[1]**2) + U[2]**2 + 4 * U[0] * U[1] + 2 * U[2] * (U[0] + U[1]),
     3 * (U[0]**2 + U[2]**2) + U[1]**2 + 4 * U[0] * U[2] + 2 * U[1] * (U[0] + U[2])],

    [3 * (U[0]**2 + U[1]**2) + U[2]**2 + 4 * U[0] * U[1] + 2 * U[2] * (U[0] + U[1]),
     12 * U[1]**2 + 2 * (U[0]**2 + U[2]**2 + U[0] * U[2]) + 6 * U[1] * (U[0] + U[2]),
     3 * (U[1]**2 + U[2]**2) + U[0]**2 + 4 * U[1] * U[2] + 2 * U[0] * (U[1] + U[2])],

    [3 * (U[0]**2 + U[2]**2) + U[1]**2 + 4 * U[0] * U[2] + 2 * U[1] * (U[0] + U[2]),
     3 * (U[1]**2 + U[2]**2) + U[0]**2 + 4 * U[1] * U[2] + 2 * U[0] * (U[1] + U[2]),
     12 * U[2]**2 + 2 * (U[0]**2 + U[1]**2 + U[0] * U[1]) + 6 * U[2] * (U[0] + U[1])]
    ]).reshape(3,3)
    
    M = (1/120)*np.linalg.det(Aux)*linear_nonlinear_matrix
    
    return M


def FEM_solver(pts,tri,Leftind,LeftVal,Rightind,RightVal,Downind,DownVal,Upind,UpVal,iterations,q):
    
    eps = 1/100
    
    #Mesh data
    Npts = np.size(pts,axis=0)
    Ntri = np.size(tri,axis=0)
    
    #Dirchlet boundary conditions and free nodes
    Dir_bdry = np.hstack((Leftind,Rightind,Downind,Upind))
    free_nodes = np.setdiff1d(np.arange(0,Npts,1), Dir_bdry, assume_unique=True)
    free_nodes_t = np.array(free_nodes)[:,None]
    
    #Imposing Dirchlet boundary conditions
    q[Leftind] = LeftVal
    q[Rightind] = RightVal
    q[Downind] = DownVal
    q[Upind] = UpVal
            
    soln = []
    soln.append(q.copy())
    
    #Newton-Raphson iteration
    for i in range(iterations):
        
        #Initializing matrices, vectors that vary throughout the algorithm
        DJ = csr_matrix((Npts,Npts), dtype = np.float).toarray()   #Jacobian of Non-linear functional matrix
        J = csr_matrix((Npts,1), dtype = np.float).toarray()       #Non-linear functional column vector
        b = np.zeros((Npts,1))                                     #load vector
        
        #Assembling non-linear functional and the Jacobian matrix of the non-linear functional
        for j in range(Ntri):

            #Triangle information
            verts = pts[tri[j,:],:] # vertices of mesh triangle
            vmid = np.reshape(np.sum(verts,axis=0)/3,(1,2)) # midpoint of mesh triangle
            ind = tri[j,:]
            indt = np.array(ind)[:,None]

            #value of U at the vertices of the triangle
            U = q[ind]
            
            #Assembling the Jacobian matrix of the non-linear functional
            DJ[indt,ind] = DJ[indt,ind]  + eps*stima3(verts) - massma3(verts) + ucube_linear(verts, U)
            
            #Assembling non-linear functional column vector
            J[ind] = J[ind] + eps*stima3(verts)@U - massma3(verts)@U + ucube(verts, U)
            
            #Assembling load vector/Volume Forces
            #Constant f = 1 is assumed. For more general f, needs to be defined outside the for loop
            #Commented out since there are no volume forces
            #Aux = np.ones((3,3))
            #Aux[1:3,:] = np.transpose(v)
            #b[indt] = b[indt] + (1/6)*np.linalg.det(Aux) 
            
            #Neumann conditions
            #None in our example!
        
        #Determine free nodes not constrained by Dirichlet boundary conditions
        A = DJ[free_nodes_t,free_nodes]
        c = J[free_nodes]
        
        #Single Newton step
        w = np.zeros((Npts,1))
        #Imposing Dirchlet boundary conditions on w
        w[Leftind] = LeftVal
        w[Rightind] = RightVal
        w[Downind] = DownVal
        w[Upind] = UpVal
        #Solving linear system
        w[free_nodes] = scipy.linalg.solve(A, c)
        
        q = q - w
        
        soln.append(q.copy())
        
        #Termination condition
        if np.linalg.norm(w) < 1e-10:
            break
    
    return soln